# Fake news Classifier using LSTM and Word Embedding

## Importing the Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Data Cleaning

In [4]:
# Dropping NaN values
df = df.dropna()

In [5]:
# Getting independent feature
X = df.drop('label',axis=1)

In [6]:
# Getting dependent features
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.2.0'

In [11]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

In [13]:

voc_size = 5000

## one-hot Representation

In [14]:
messages = X.copy()

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

### Data Preprocessing

In [17]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    print(i)
    review = re.sub('[^a-zA-z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

1
14952
14953
14954
14955
14956
14957
14958
14959
14960
14961
14962
14963
14964
14965
14966
14967
14968
14969
14970
14971
14972
14973
14974
14975
14976
14977
14978
14979
14980
14981
14982
14983
14984
14985
14986
14987
14988
14989
14990
14991
14992
14993
14994
14995
14996
14997
14998
14999
15000
15001
15002
15003
15004
15005
15006
15007
15008
15009
15010
15011
15012
15013
15014
15015
15016
15017
15018
15019
15020
15021
15022
15023
15024
15025
15026
15027
15028
15029
15030
15031
15032
15033
15034
15035
15036
15037
15038
15039
15040
15041
15042
15043
15044
15045
15046
15047
15048
15049
15050
15051
15052
15053
15054
15055
15056
15057
15058
15059
15060
15061
15062
15063
15064
15065
15066
15067
15068
15069
15070
15071
15072
15073
15074
15075
15076
15077
15078
15079
15080
15081
15082
15083
15084
15085
15086
15087
15088
15089
15090
15091
15092
15093
15094
15095
15096
15097
15098
15099
15100
15101
15102
15103
15104
15105
15106
15107
15108
15109
15110
15111
15112
15113
15114
15115
15116
15117
15

In [18]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]

## Embedding Representation


In [19]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

## Model Creation

In [20]:
embedded_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:
import numpy as np

In [24]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=0)

## Model Training

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 24ms/step - loss: 0.3225 - accuracy: 0.8472 - val_loss: 0.1976 - val_accuracy: 0.9155
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1506 - accuracy: 0.9432 - val_loss: 0.1909 - val_accuracy: 0.9236
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1094 - accuracy: 0.9631 - val_loss: 0.2049 - val_accuracy: 0.9231
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0898 - accuracy: 0.9672 - val_loss: 0.2198 - val_accuracy: 0.9183
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0669 - accuracy: 0.9778 - val_loss: 0.2478 - val_accuracy: 0.9225
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0477 - accuracy: 0.9834 - val_loss: 0.3090 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0403 - accuracy: 0.9873 - val_loss: 0.2771 - val_accuracy: 0.9162

In [28]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3147,  242],
       [ 267, 2379]], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9156586578293289

In [31]:
model.save('model.h5')